In [0]:
# All functions needed for bruteswitching

import math as m

def CreateBlockMatrix(dimensions, blockshape, blocklist):
    '''Create one matrix out of several specified blocks'''
    return block_matrix(dimensions[0],dimensions[1], [blocklist[t] for t in blockshape])



def is_zeroone(matrix):
    '''Check if a matrix is a zero-one matrix'''
    for row in matrix:
        for entry in row:
            if entry not in {0,1}:
                return False

    return True



def switchingblock(level, blocknumber, sort):
    '''Create a regular orthogonal permutation-based matrix of a certain level, number of blocks and sort'''
    J = ones_matrix(level)
    O = zero_matrix(level)
    Y = level*identity_matrix(level)- J

    def rotate(l):
        return [l[-1]] + l[:-1]

    baserow = [0,2] + [1]*(blocknumber-2)
    shapematrix = []
    for _ in range(blocknumber):
        shapematrix = shapematrix + baserow
        baserow = rotate(baserow)


    if sort == 'wqh':
        return CreateBlockMatrix([blocknumber,blocknumber], shapematrix, [Y,O,J])/level
    if sort == 'gm':
        return CreateBlockMatrix([blocknumber,blocknumber], shapematrix, [-Y,O,J])/level



def getvertices(n, S):
    '''Gives a list where the numbers in S are moved to the front, which will be used as a permutation'''
    complement = set(range(1,n+1)) - set(S)
    return S + list(complement)



def symmetrymatrices(n, blocks, level, sort, recursionlevel):
    '''This function recursively finds all permutations (as matrices) within the switching set.
    At each recursion level it takes any set of level size and puts it to the front. We do this, because the switching matrix has a symmetry group induced by rotating the blocks and rotating within the blocks. Therefore it only matters what the set of vertices in each block are, so we pick combinations. First we pick the second to last block, then at each recursion level we go back until we pick the first block.For each block we may only pick from [1,....,blocks*level-1]. Thus ensuring that the last block has the highest number in it. This beats the symmetry of the block rotation.'''

    #I could remove n and replace by blocks*level just have to add identity matrix to P1
    if blocks == 2 and sort == 'gm':
        return [identity_matrix(n)]
    if recursionlevel == 1:
        return [identity_matrix(n)]

    matrices= []
    for X in Combinations(range(level*(blocks-recursionlevel)+1, blocks*level), level):
        Xmatrix = Permutation(getvertices(n, X)).to_matrix()
        for P in symmetrymatrices(n, blocks, level, sort, recursionlevel-1):
            # As the matrices will be transposed (on the left) in bruteswitching. The way to ensure that the highest level does the second to last block is to multiply Xmatrix on the left. To be honest I don't think it matters, you will get the same set of symmetrymatrices either way, but at least this way I can explain it.
            matrices.append(Xmatrix*P)

    return matrices





def Bruteswitching(gen, n, level, blocks, sort):
    '''Check all graphs in gen to see if they have a switching set of type sort and size blocks*level
    Multiple things get counted along the way.

    counttotal checks the total number of graphs we iterate over, so we can check that it matches the amount of graphs that we know  should be in gen.
    countswitchingsets counts the number of sets among all graphs in gen that we can perform the studied switching ona. Note that for WQH etc. we check all different ways to split into C_1 and C_2 by using symmetrymatrices().
    countcospectral counts per graph the number of cospectral mates through the studied switching.
    If a graph has a cospectral mate then it gets added to the switchgraphs list.'''

    switchgraphs = []
    counts = []
    counttotal = 0
    countswitchingsets = 0

    Q = switchingblock(level, blocks, sort).block_sum(identity_matrix(n-blocks*level))
    Permlist = symmetrymatrices(n, blocks, level, sort, blocks)
    Potential_switchingsets = Combinations(range(1,n+1),blocks*level)
    
    for G in gen:
        counttotal += 1
        if G is None:
            continue
        countcospectral = 0
        A = G.adjacency_matrix()


        for S in Potential_switchingsets:
            P0 = Permutation(getvertices(n,S)).to_matrix()
            for P1 in Permlist:
                P = P0*P1
                B = Q.transpose()*P.transpose()*A*P*Q
                if is_zeroone(B):
                    countswitchingsets +=1
                    if not Graph(B).is_isomorphic(G):
                        countcospectral += 1


        if countcospectral > 0:
            switchgraphs.append(G)
            counts.append(countcospectral)

    # print(f'checked {counttotal} graphs')
    # print(f'Among these there were {countswitchingsets} switching sets')
    return switchgraphs, counts


In [0]:
import time
import math as m

def enumerateCospectralgraphs(n, level, blocks, sort):
    cospectralgraphs = []
    enumerateCospectral = []
    allcospectral = []
    start_time = time.time()
    for k in range(m.floor(n*(n-1)/4)+1):
        allcospectral.append([])
        for x in graphs.cospectral_graphs(n, graphs=lambda x: len(x.edges())==k):
            allcospectral[k] = allcospectral[k] + x

        cospectralgraphs.append(Bruteswitching(allcospectral[k], n, level, blocks, sort)[0])
    
    numbercospectral = [len(cospectralbysize) for cospectralbysize in cospectralgraphs]
    if (n % 4) in {0,1}:
        enumerateCospectral.append(2*sum(numbercospectral[:-1])+ numbercospectral[-1])
    else:
        enumerateCospectral.append(2*sum(numbercospectral))
    print(f'time for {n} vertices', time.time() - start_time)

    return enumerateCospectral, numbercospectral

#save list of 
def enumerateSwitching(n, level, blocks, sort, potentiallist):
    return len(Bruteswitching(potentiallist, n, level, blocks, sort))


Allswitchingsenumerate = [[],[],[],[],[],[]]
for n in range(9,10):
    continue
    # Allswitchingsenumerate[0].append(enumerateCospectralgraphs(n, 2, 2, 'gm'))
    # Allswitchingsenumerate[1].append(enumerateCospectralgraphs(n, 3, 2, 'gm'))
    # Allswitchingsenumerate[2].append(enumerateCospectralgraphs(n, 4, 2, 'gm'))
    # Allswitchingsenumerate[3].append(enumerateCospectralgraphs(n, 3, 2, 'wqh'))
    # Allswitchingsenumerate[4].append(enumerateCospectralgraphs(n, 4, 2, 'wqh'))
    #Allswitchingsenumerate[5].append(enumerateCospectralgraphs(n, 2, 3, 'wqh'))

    
print(Allswitchingsenumerate)

In [0]:
import itertools as its

def symmetryreduce(list_of_edges, matrix):
    Aut = Graph(matrix).automorphism_group()

    list_of_representatives = []
    set_of_edges_in_orbits = []
    for M in list_of_edges:
        if M not in set_of_edges_in_orbits:
            list_of_representatives.append(M)
            for P in Aut:
                set_of_edges_in_orbits.append(M*Permutation(P).to_matrix())
    
    return list_of_representatives

    
def BuildGraphsWithSwitching(outersize, switchingsize, outergraph, switchingsets, valid_adjacencies, first_column):
    allgraphswithswitching = []
    if outersize > 0:
        allvalid_edges = [matrix(outersize-1, switchingsize, M) for M in its.product(valid_adjacencies, repeat=outersize-1)]
    else:
        return switchingsets
    
    #somehow add dict A: symmetry_adj
    for A in switchingsets:
        for W in first_column:
            for V in allvalid_edges:
                V = V.stack(matrix(1, switchingsize, W))
                switchingmatrix = A.augment(V.transpose())
                for C in outergraph:
                    bottommatrix= V.augment(C)
                    allgraphswithswitching.append(switchingmatrix.stack(bottommatrix))
        
    return allgraphswithswitching

def reducelisttoiso(graphlist):
    immutablegraphs = [G.copy(immutable=True) for G in graphlist]
    return set(immutablegraphs)

        
def checkcospectralmate(Q, graphswithswitching):
    hascospectralmate = []
    for A in graphswithswitching:
        Bcanlabel =  Graph(Q.transpose()*A*Q).canonical_label()
        Acanlabel = Graph(A).canonical_label()
        if Bcanlabel != Acanlabel:
            hascospectralmate += [Acanlabel, Bcanlabel, Acanlabel.complement(), Bcanlabel.complement()]
    
    print(len(hascospectralmate))
    return reducelisttoiso(hascospectralmate)

# def checkcospectral2(Q, graphswithswitching):
#     hascospectralmate = []
#     for A in graphswithswitching:
#         B =  Q.transpose()*A*Q
#         G = Graph(A)
#         if not Graph(B).is_isomorphic(G):
#             hascospectralmate.append(G)
#             hascospectralmate.append(G.complement())
    
#     print(len(hascospectralmate))
#     return reducelisttoiso2(hascospectralmate)

# def reducelisttoiso2(graphlist):
#     reduced = []
#     for G in graphlist:
#         new = True
#         for H in reduced:
#             if H.is_isomorphic(G):
#                 new = False
#                 break
#         if new:
#             reduced.append(G)

#     return reduced

def Genswitchingset(level, blocks, sort):
    '''Create all labelled graphs (up to complementation) that form a switching set for the switching with given blocks, level and sort 
        up to permutation that leaves Q fixed.'''
    switchingset = []

    Q = switchingblock(level, blocks, sort)
    Permlist = symmetrymatrices(blocks*level, blocks, level, sort, blocks)
    n = level*blocks
    for G in graphs.nauty_geng(str(n) + ' 0:' + str(m.floor(n*(n-1)/4))):
        A = G.adjacency_matrix()
        for P in Permlist:
            B = Q.transpose()*P.transpose()*A*P*Q
            if is_zeroone(B):
                switchingset.append(P.transpose()*A*P)

    return [k for k, _ in its.groupby(sorted(switchingset))]

def GoodEdges(level, number_of_blocks, sort):
    '''Create a set of vectors that satisfy the conditions for a permutation family switching'''
    goodedges = []
    for v in its.product(range(2), repeat = level*number_of_blocks):
        sumsperblock = []
        for i in range(number_of_blocks):
            sumsperblock.append(sum(v[i*level:level*(i+1)]) % level)

        if sort == 'wqh':        
            if len(set(sumsperblock)) == 1:
                goodedges.append(v)

        elif sort == 'gm':
            should_append = True
            for i in range(number_of_blocks):
                if (sumsperblock[i] + sumsperblock[(i+1) % number_of_blocks]) % level != 0:
                    should_append = False

            if should_append:
                goodedges.append(v)

    return goodedges



In [0]:
level = 4
sort = 'wqh'
numberblocks = 2


goodedges = GoodEdges(level, numberblocks, sort)
switchingsets = Genswitchingset(level, numberblocks, sort)
print(len(switchingsets))

reduceswitchcolumn = []
firstandswitched = []
Q = switchingblock(level, numberblocks, sort)
for x in goodedges:
    y = vector(x)
    if y not in firstandswitched:
        firstandswitched.append(y)
        firstandswitched.append(y*Q)
        reduceswitchcolumn.append(y)

# firstcol = dict()
# for struct in switchingsets:
#     for x in first_column:

#     firstcol[struct] = 
# symmetries_of_Q = PermutationGroup([(1,2)], [(1,3)], [(1,4)], [(5,6)], [(5,7)], [(5,8)], [(1,5),(2,6),(3,7),(4,8)])

# def switchingaut(columnvec, switchingset, symmetryorthog):

#     automorphisms = []
#     for g in symmetryorthog:
#         P = g.to_matrix()
#         if P.transpose()*switchingset*P:
#             automorphisms.append(P)
    
#     orbits = []
#     vectorsuptosym = []
#     for v in columnvec:
#         if v not in orbits:
#             vectorsuptosym.append(v)
#             for P in automorphisms:
#                 orbits.append(v*P)

#     return vectorsuptosym

# last_column = switchingaut(reduceswitchcolumn, )

last_column = reduceswitchcolumn
##############################################
added_vertices = 0

HasAH = BuildGraphsWithSwitching(added_vertices, numberblocks*level, [G.adjacency_matrix() for G in graphs(added_vertices)], switchingsets, goodedges, last_column)
print(len(HasAH))

Q = switchingblock(level, numberblocks, sort)
Q = Q.block_sum(identity_matrix(added_vertices))
Graphswithmates = checkcospectralmate(Q, HasAH)
print(len(Graphswithmates))



next file
